In [1]:
import pandas as pd

In [2]:
#read in account data
clipper_accounts = pd.read_csv("~/Box/USF_Data_Science_Practicum/Analytics/Anonymous_Data/Clipper_Account_Holders_Anonymous.csv", encoding = 'utf-8')
fastrak_accounts = pd.read_csv("~/Box/USF_Data_Science_Practicum/Analytics/Anonymous_Data/FasTraK_Profile_Anonymous.csv", encoding = 'utf-8')

## Clipper Exploration

In [51]:
clipper_accounts.head(3)

,CONTACT_ID,CITY,STATE,CARD_SERIAL_NUMBER,CARD_TYPE,STATUS,FIRST_NAME,LAST_NAME
0,0000000000246131,\tBerkeley,CA,1205337096,Senior Citizen,Active,6b989ea713f67acc257f561c7402d8b7cfa5810d6a9893...,e5304704dcc182c23a8d161ef92262145c98a7add8c6d8...
1,0000000000277DF2,San Rafael,CA,1202594132,Senior Citizen,Active,ba79f7ea69056fd0668d0eb9267ded5f47d42725d8b5cb...,a56cd08be93a66bba4a5955de4f5cd0241c0f14e75616b...
2,00000000002265B4,San Francisco,CA,1204075258,Adult,Active,df37365766ae084a53f1117894efacd6255edb45272478...,bb88d95ef1e7929110cc442fb155d8756aea00a759b6f0...


In [52]:
#number rows, columns in df
clipper_accounts.shape

(2482848, 8)

In [53]:
#rename columns
clipper_accounts =clipper_accounts.rename(index=str, columns={"STATE_": "STATE",
                                                              "FN_Anonymous": "FIRST_NAME",
                                                             "LN_Anonymous": "LAST_NAME"})
clipper_accounts.head(3)

,CONTACT_ID,CITY,STATE,CARD_SERIAL_NUMBER,CARD_TYPE,STATUS,FIRST_NAME,LAST_NAME
0,0000000000246131,\tBerkeley,CA,1205337096,Senior Citizen,Active,6b989ea713f67acc257f561c7402d8b7cfa5810d6a9893...,e5304704dcc182c23a8d161ef92262145c98a7add8c6d8...
1,0000000000277DF2,San Rafael,CA,1202594132,Senior Citizen,Active,ba79f7ea69056fd0668d0eb9267ded5f47d42725d8b5cb...,a56cd08be93a66bba4a5955de4f5cd0241c0f14e75616b...
2,00000000002265B4,San Francisco,CA,1204075258,Adult,Active,df37365766ae084a53f1117894efacd6255edb45272478...,bb88d95ef1e7929110cc442fb155d8756aea00a759b6f0...


In [21]:
#number unique values per column
clipper_accounts.T.apply(lambda x: x.nunique(), axis=1)

CONTACT_ID            1678394
CITY                     4116
STATE_                      1
CARD_SERIAL_NUMBER    2482848
CARD_TYPE                  17
STATUS                      2
FN_Anonymous           247136
LN_Anonymous           296400
dtype: int64

### City

In [24]:
#number of unique cities
clipper_accounts['CITY'].nunique()

4116

In [32]:
#what values of city do we have
clipper_accounts['CITY'].value_counts()

San Francisco          743887
Oakland                241791
San Jose               113347
Berkeley                97100
Rtcity                  95031
Fremont                 59347
Alameda                 45840
Walnut Creek            37041
Sf                      36057
Hayward                 34477
Daly City               33714
Concord                 32680
Richmond                31035
San Leandro             29284
San Mateo               27849
Sunnyvale               26960
Nocity                  23252
Mountain View           21014
San Rafael              20801
Palo Alto               19030
Redwood City            18112
San Francisco           17770
Pleasanton              17443
Santa Clara             16886
South San Francisco     16800
El Cerrito              16108
San Ramon               15539
Union City              15533
Castro Valley           14616
Dublin                  13799
                        ...  
Ketfield                    1
San Jose Ave                1
Ranch Mira

In [124]:

#clipper_accounts[clipper_accounts['CARD_TYPE'].str.contains("Rtc")==True]

#### Cleaning City

Bellow, we are going to use a file from thi [Github Project](https://github.com/grammakov/USA-cities-and-states), which contains all the cities in US

In [3]:
cities = pd.read_csv("~/Box/Data/us_cities_states_counties.csv", encoding = 'utf-8', sep = "|")

In [4]:
# Taking only the cities in California
california_cities = cities[cities["State full"] == "California"]

In [6]:
# Creating a list of all the unique cities on CA
list_of_cities = california_cities["City"].unique()
# Turning all of them to lowercase
list_of_cities = map(lambda x:x.lower(), list_of_cities)

Now it is time to clean some of the Clipper data, for the City column

In [7]:
# Turning all of them to lowercase
clipper_accounts["CITY"] = clipper_accounts["CITY"].str.strip().str.lower()
# Replace some elemnts that are very frequently misspelled
clipper_accounts["CITY"] = clipper_accounts["CITY"].replace("sf", "san francisco")
clipper_accounts["CITY"] = clipper_accounts["CITY"].replace("af", "san francisco")
clipper_accounts["CITY"] = clipper_accounts["CITY"].replace("la", "los angeles")
clipper_accounts["CITY"] = clipper_accounts["CITY"].str.replace("  ", " ")

In [8]:
clipper_accounts["CITY"].value_counts()

san francisco          798187
oakland                246202
san jose               116345
berkeley                99501
rtcity                  95031
fremont                 60297
alameda                 46518
walnut creek            37860
hayward                 35262
daly city               35019
concord                 33243
richmond                32012
san leandro             30157
san mateo               28681
sunnyvale               27627
nocity                  23252
mountain view           21397
san rafael              21183
palo alto               19449
redwood city            18729
pleasanton              17775
santa clara             17387
south san francisco     17344
el cerrito              16479
union city              15829
san ramon               15816
castro valley           14884
dublin                  13986
antioch                 13947
albany                  13519
                        ...  
sn juan batsta              1
cornell                     1
daly city 

Bellow, we created a function, which replaces misspelled city names with the correct ones

In [9]:
def is_similar(first):
    
    # First get the best score for the given city
    score = 0.0
    for s in list_of_cities:
        if difflib.SequenceMatcher(None, first, s).ratio() > score:
            score = difflib.SequenceMatcher(None, first, s).ratio()
            city = s
    # If the score is lower then a threshold of 0.7 then return the original city name
    # which means we did not find a good representation for the city
    if score < 0.7:
        city = first
    # Otherwise it returns the one with the best score
    return city

In [ ]:
clipper_accounts["CITY"] = clipper_accounts["CITY"].apply(is_similar)

In [ ]:
clipper_accounts["CITY"].value_counts()

### State

In [54]:
#number of unique States
clipper_accounts['STATE'].nunique()

1

### Contact Id

In [26]:
#number of unique contact ids
clipper_accounts['CONTACT_ID'].nunique()

1678394

In [39]:
#look at rows with duplicated CONTACT_ID
clipper_accounts[clipper_accounts.CONTACT_ID.duplicated(keep=False)].head(10)

,CONTACT_ID,CITY,STATE_,CARD_SERIAL_NUMBER,CARD_TYPE,STATUS,FN_Anonymous,LN_Anonymous
7,0000000000140F88,Antioch,CA,1206496473,Adult,Active,684645677c208dfdb3999bfbf5773b3f43183488e984bd...,649be9a47800da614e5093cee0b882e9070af9aae4b59c...
8,0000000000140F88,Antioch,CA,1204727228,Adult,Blocked,684645677c208dfdb3999bfbf5773b3f43183488e984bd...,649be9a47800da614e5093cee0b882e9070af9aae4b59c...
9,0000000000140F88,Antioch,CA,1203173627,Adult,Blocked,684645677c208dfdb3999bfbf5773b3f43183488e984bd...,649be9a47800da614e5093cee0b882e9070af9aae4b59c...
22,00000000000B15A7,Berkeley,CA,1206653374,Adult,Active,39ca22d05e9a10c61df66e3589d2da95d8335b6ef0f76c...,60bd1c0f36870f6cae6a4989586aadd1725a9d0db6a369...
23,00000000000B15A7,Berkeley,CA,1204305305,Adult,Blocked,39ca22d05e9a10c61df66e3589d2da95d8335b6ef0f76c...,60bd1c0f36870f6cae6a4989586aadd1725a9d0db6a369...
24,00000000000B15A7,Berkeley,CA,1201798200,Adult,Blocked,39ca22d05e9a10c61df66e3589d2da95d8335b6ef0f76c...,60bd1c0f36870f6cae6a4989586aadd1725a9d0db6a369...
25,00000000000B15A7,Berkeley,CA,1200544331,Adult,Blocked,39ca22d05e9a10c61df66e3589d2da95d8335b6ef0f76c...,60bd1c0f36870f6cae6a4989586aadd1725a9d0db6a369...
30,0000000000227356,Berkeley,CA,1205904058,Adult,Active,bfdc74a5840acc4f1e53edd0d28101d2ec5270511529d2...,0a05a01671e751ed5201687193343b0aff191ec82e4733...
31,0000000000227356,Berkeley,CA,1205792360,Adult,Active,bfdc74a5840acc4f1e53edd0d28101d2ec5270511529d2...,0a05a01671e751ed5201687193343b0aff191ec82e4733...
38,0000000000264FF6,Burbank,CA,1204832571,Adult,Active,e4523c2e444f2937ab7dcd6afb004a86013f4b89e2c42a...,f5ce9ba8e2f347e8de89ce436d30ba37fe326609f2f015...


The above shows that there can be multiple cards linked to one account. Usually all but 1 are blocked, but that is not always the case.

### Card Type

In [43]:
#number of unique cardtype
clipper_accounts['CARD_TYPE'].nunique()

17

In [44]:
#what values of cardtype do we have
clipper_accounts['CARD_TYPE'].value_counts()

Adult                       1646616
Senior Citizen               338726
Youth                        267914
Rtc Discount                 223700
Tsb Non - Fare Card            1980
Initialized                    1154
Bart Non - Fare Card            960
Tsb Non-Fare Card               847
Rtc Desfire                     489
Sf Muni Non - Fare Card         146
Ggt Non - Fare Card              88
Vta Non - Fare Card              85
Act Non - Fare Card              44
Non-Fare                         37
Failed                           32
Adult - Obsolete                 16
Caltrain Non - Fare Card         14
Name: CARD_TYPE, dtype: int64

### Status

In [45]:
#number of unique cardtype
clipper_accounts['STATUS'].nunique()

2

In [46]:
#what values of status do we have
clipper_accounts['STATUS'].value_counts()

Active     1932789
Blocked     550059
Name: STATUS, dtype: int64

### Names

In [55]:
#number of unique first names
clipper_accounts['FIRST_NAME'].nunique()

247136

In [56]:
#number of unique last names
clipper_accounts['LAST_NAME'].nunique()

296400

In [58]:
#number of unique first-last name combo's
(clipper_accounts['FIRST_NAME'].map(str) + clipper_accounts['LAST_NAME']).nunique()

1370024

In [60]:
# num unique accounts minus num unique first/last name combos
clipper_accounts['CONTACT_ID'].nunique() - (clipper_accounts['FIRST_NAME'].map(str) + clipper_accounts['LAST_NAME']).nunique()

308370

In [91]:
#only keep one row per CONTACT_ID
clipper_acct_dedup = clipper_accounts.drop_duplicates(subset='CONTACT_ID', keep='first', inplace=False)

#concat first and last name
clipper_acct_dedup["FIRST_LAST"] = clipper_acct_dedup[['FIRST_NAME', 'LAST_NAME']].apply(lambda x: '_'.join(x), axis=1)

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [93]:
#look at rows with duplicated FIRST_LAST (after duplicated account removed)
clipper1 = clipper_acct_dedup[clipper_acct_dedup.FIRST_LAST.duplicated(keep=False)].sort_values('FIRST_LAST')
print clipper1.shape
clipper1.head(6)

(459186, 9)


,CONTACT_ID,CITY,STATE,CARD_SERIAL_NUMBER,CARD_TYPE,STATUS,FIRST_NAME,LAST_NAME,FIRST_LAST
445847,000000000004C88F,Fremont,CA,1200442905,Adult,Active,00026138b6ae717197cd9b99dccb2160a6cba0f438dfee...,527d0da3ed4f804bb25c1f9e1569d6864af39924efdb29...,00026138b6ae717197cd9b99dccb2160a6cba0f438dfee...
447515,000000000023BDA8,Fremont,CA,1206359523,Adult,Active,00026138b6ae717197cd9b99dccb2160a6cba0f438dfee...,527d0da3ed4f804bb25c1f9e1569d6864af39924efdb29...,00026138b6ae717197cd9b99dccb2160a6cba0f438dfee...
2221363,000000000014548C,San Pablo,CA,1203633844,Youth,Active,00056bcae7f0d301859fb23c2ceaa5bed10d50ff3aec89...,492c473f67214f8a12d5a3eb781c4489de7f6eb69c4af8...,00056bcae7f0d301859fb23c2ceaa5bed10d50ff3aec89...
1118327,00000000002688C3,Richmond,CA,1206422396,Adult,Active,00056bcae7f0d301859fb23c2ceaa5bed10d50ff3aec89...,492c473f67214f8a12d5a3eb781c4489de7f6eb69c4af8...,00056bcae7f0d301859fb23c2ceaa5bed10d50ff3aec89...
1881458,0000000000274070,San Francisco,CA,1207282348,Adult,Active,000d8ec299b67fbdecfb8b3d43296fa04a4e2585b1008e...,6bc36c0c73df7c89bfc8916c9220addbd2bf89c9b167c1...,000d8ec299b67fbdecfb8b3d43296fa04a4e2585b1008e...
1110019,000000000009A729,Richmond,CA,1201451839,Youth,Active,000d8ec299b67fbdecfb8b3d43296fa04a4e2585b1008e...,6bc36c0c73df7c89bfc8916c9220addbd2bf89c9b167c1...,000d8ec299b67fbdecfb8b3d43296fa04a4e2585b1008e...


Because there are 400K+ unique accounts with duplicated first/last name combos, we will need multiple fields to match on

## Fastrak Exploration

In [94]:
#only keep one row per CONTACT_ID
fastrak_accounts = fastrak_accounts.drop_duplicates(subset='Account', keep='first', inplace=False)

In [95]:
#number rows, columns in df
fastrak_accounts.shape

(2141006, 10)

In [103]:
fastrak_accounts = fastrak_accounts.rename(index=str, columns={"FN_ANONYMOUS": "FIRST_NAME",
                                                             "LN_Anonymous": "LAST_NAME"})
fastrak_accounts.head(3)

,Account,Account_Type,Account_Status,Account_Opening_Data,City,STATE,Device,Device Status,FIRST_NAME,LAST_NAME
0,1000003,Private,Active,29-DEC-99,Hercules,CA,903640519,Active,e3c795cbb55dbf0a776ca6c2b57f54112909a3de456905...,7c5f2e1df53a5cbcf6ac9c4c60d488e9a8fe9af3451d01...
1,1000006,Private,Active,05-JAN-00,Benicia,CA,2609100650,Active,63a912989826f2f4c3f8770424a427cc52c9a1637af3c2...,60781dec202e45a49f8b1d8ba030f78e884f80ec3ec0a5...
4,1000009,Private,Active,05-JAN-00,Hayward,CA,419800377,Swapnoreturn,e34c463efa273e07883b01ed78a714e904a9eef856052c...,4b1731d0a8f9a5edca3a2aa4f81110ef86687b0c933848...


In [104]:
#unique values per column
fastrak_accounts.T.apply(lambda x: x.nunique(), axis=1)

Account                 2141006
Account_Type                  2
Account_Status                6
Account_Opening_Data       6511
City                       3647
STATE                         1
Device                  2141006
Device Status                15
FIRST_NAME               182248
LAST_NAME                330248
dtype: int64

### City

In [28]:
#number of unique cities
fastrak_accounts['City'].nunique()

3647

In [33]:
#what values of city do we have
fastrak_accounts['City'].value_counts()

San Francisco            389550
Oakland                  171991
San Jose                  96583
Fremont                   89930
Sacramento                74206
Hayward                   72721
Vallejo                   65299
Santa Rosa                61207
Berkeley                  57991
San Rafael                54928
Concord                   52525
Walnut Creek              52025
Fairfield                 46914
San Ramon                 44724
San Mateo                 44492
Richmond                  43257
Livermore                 41671
San Leandro               41097
Novato                    40273
Pleasanton                40233
Alameda                   39896
Danville                  38269
Daly City                 36058
Vacaville                 34588
Redwood City              34206
Petaluma                  32882
Mill Valley               29409
Union City                29079
Dublin                    28620
Benicia                   28493
                          ...  
Woodridg

,CONTACT_ID,CITY,STATE,CARD_SERIAL_NUMBER,CARD_TYPE,STATUS,FIRST_NAME,LAST_NAME


### State

In [41]:
#number of unique States
fastrak_accounts['STATE'].nunique()

1

### Account Status

In [99]:
fastrak_accounts['Account_Status'].value_counts()

Active        1863343
Closed         181400
Rvkf            60153
Collection      27210
Rvkw             8377
Close Pend        523
Name: Account_Status, dtype: int64

### Account Type

In [100]:
fastrak_accounts['Account_Type'].value_counts()

Private     2090236
Business      50770
Name: Account_Type, dtype: int64

### Names

In [105]:
#number of unique first names
fastrak_accounts['FIRST_NAME'].nunique()

182248

In [106]:
#number of unique last names
fastrak_accounts['LAST_NAME'].nunique()

330248

In [107]:
#number of unique first-last name combo's
(fastrak_accounts['FIRST_NAME'].map(str) + fastrak_accounts['LAST_NAME']).nunique()

1635307

In [109]:
# num unique accounts minus num unique first/last name combos
fastrak_accounts.shape[0] - (fastrak_accounts['FIRST_NAME'].map(str) + fastrak_accounts['LAST_NAME']).nunique()


505699

In [118]:
#concat first and last name
fastrak_accounts["FIRST_LAST"] = fastrak_accounts[['FIRST_NAME', 'LAST_NAME']].apply(lambda x: '_'.join(x), axis=1)

In [119]:
#look at rows with duplicated FIRST_LAST (after duplicated account removed)
fastrak1 = fastrak_accounts[fastrak_accounts.FIRST_LAST.duplicated(keep=False)].sort_values('FIRST_LAST')
print fastrak1.shape
fastrak1.head(6)

(723632, 11)


,Account,Account_Type,Account_Status,Account_Opening_Data,City,STATE,Device,Device Status,FIRST_NAME,LAST_NAME,FIRST_LAST
2719163,154468525,Private,Active,23-AUG-16,San Francisco,CA,1468110777,Active,000737810dcbb4180694433a7f49d4d4ee120730c8f556...,e44b24ce28fe70a4e3036c7ed2c5201a9639ae3dea4bab...,000737810dcbb4180694433a7f49d4d4ee120730c8f556...
1222981,115208580,Private,Closed,29-FEB-12,Richmond,CA,1451990234,Lost,000737810dcbb4180694433a7f49d4d4ee120730c8f556...,e44b24ce28fe70a4e3036c7ed2c5201a9639ae3dea4bab...,000737810dcbb4180694433a7f49d4d4ee120730c8f556...
2846217,157201261,Private,Active,26-JAN-17,Hayward,CA,1472010446,Active,000a0812f235d3ed6ad90976b4bff69e7a1c62e8decdac...,efa037bdff4adcde1eefdecf574728f7e2798e24ddfd8c...,000a0812f235d3ed6ad90976b4bff69e7a1c62e8decdac...
2818343,156665765,Private,Rvkf,23-DEC-16,Hayward,CA,1468840128,Revoked,000a0812f235d3ed6ad90976b4bff69e7a1c62e8decdac...,efa037bdff4adcde1eefdecf574728f7e2798e24ddfd8c...,000a0812f235d3ed6ad90976b4bff69e7a1c62e8decdac...
2824622,156788244,Private,Active,29-DEC-16,Lafayette,CA,1472020671,Active,0010c05b1de3149acc37244d67794c3d086155469d66b5...,0f31affe3b00617e02744406a4195d2023a6256615cc7b...,0010c05b1de3149acc37244d67794c3d086155469d66b5...
1188176,114665989,Private,Closed,10-OCT-11,San Francisco,CA,1451650585,Lost,0010c05b1de3149acc37244d67794c3d086155469d66b5...,0f31affe3b00617e02744406a4195d2023a6256615cc7b...,0010c05b1de3149acc37244d67794c3d086155469d66b5...


Will need to look at if some of these with same first and last name but diff account numbers are actually same person